In [ ]:
# 상관계수 만들기 OIL PRESENT, 나프타, 황중유, 

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [3]:
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv("화이팅_ㅋ.csv")

In [3]:
df

,주차,SCFI,코로나확진자,환율,WTI유가,나프타유가,고유황중유가
0,2013-1,1148.08,0.0,1056.49,93.56,106.6980,96.1060
1,2013-2,1232.35,0.0,1058.35,95.56,105.0360,97.7760
2,2013-3,1245.84,0.0,1075.58,95.88,103.4800,97.2140
3,2013-4,1227.84,0.0,1095.19,97.77,106.6620,97.4920
4,2013-5,1219.39,0.0,1096.00,95.72,108.1780,98.5660
...,...,...,...,...,...,...,...
584,2024-15,1757.04,39459.0,1374.30,83.14,74.9875,79.6475
585,2024-16,1769.54,31841.0,1376.43,83.85,74.3400,80.8120
586,2024-17,1940.63,34278.0,1354.63,78.11,75.0220,80.4920
587,2024-19,2305.79,33678.0,1351.76,80.06,71.8160,81.2640


In [4]:
# 날짜를 인덱스로 설정
df.set_index('주차', inplace=True)

In [5]:
df.iloc[563:570]

,SCFI,코로나확진자,환율,WTI유가,나프타유가,고유황중유가
주차,,,,,,
2023-52,1759.57,27732700.0,1314.16,73.81,74.1325,69.605
2023-52,1759.57,27732700.0,1314.16,73.69,74.1325,69.605
2023-52,1759.57,27732700.0,1253.50,73.81,74.1325,69.605
2023-52,1759.57,27732700.0,1253.50,73.69,74.1325,69.605
2023-52,1759.57,263390.0,1314.16,73.81,74.1325,69.605
2023-52,1759.57,263390.0,1314.16,73.69,74.1325,69.605
2023-52,1759.57,263390.0,1253.50,73.81,74.1325,69.605


In [6]:
df.drop(df.index[563:570], inplace=True)

In [7]:
df

,SCFI,코로나확진자,환율,WTI유가,나프타유가,고유황중유가
주차,,,,,,
2013-1,1148.08,0.0,1056.49,93.56,106.6980,96.1060
2013-2,1232.35,0.0,1058.35,95.56,105.0360,97.7760
2013-3,1245.84,0.0,1075.58,95.88,103.4800,97.2140
2013-4,1227.84,0.0,1095.19,97.77,106.6620,97.4920
2013-5,1219.39,0.0,1096.00,95.72,108.1780,98.5660
...,...,...,...,...,...,...
2024-15,1757.04,39459.0,1374.30,83.14,74.9875,79.6475
2024-16,1769.54,31841.0,1376.43,83.85,74.3400,80.8120
2024-17,1940.63,34278.0,1354.63,78.11,75.0220,80.4920


In [8]:
df.drop('환율', axis=1, inplace=True)

In [9]:
df

,SCFI,코로나확진자,WTI유가,나프타유가,고유황중유가
주차,,,,,
2013-1,1148.08,0.0,93.56,106.6980,96.1060
2013-2,1232.35,0.0,95.56,105.0360,97.7760
2013-3,1245.84,0.0,95.88,103.4800,97.2140
2013-4,1227.84,0.0,97.77,106.6620,97.4920
2013-5,1219.39,0.0,95.72,108.1780,98.5660
...,...,...,...,...,...
2024-15,1757.04,39459.0,83.14,74.9875,79.6475
2024-16,1769.54,31841.0,83.85,74.3400,80.8120
2024-17,1940.63,34278.0,78.11,75.0220,80.4920


In [11]:
X = df.drop(['SCFI'],axis=1)
Y = df['SCFI']

In [12]:
from keras.layers import GRU, Dense, Dropout

In [15]:
from sklearn.model_selection import train_test_split

In [13]:
# 데이터 정규화
scaler_X = RobustScaler()
scaled_X = scaler_X.fit_transform(X)

scaler_Y = RobustScaler()
scaled_Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

In [16]:
# 데이터셋을 훈련, 검증 및 테스트 세트로 분할
x_train, x_temp, y_train, y_temp = train_test_split(scaled_X, scaled_Y, test_size=0.4, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# GRU 입력 형태로 데이터 변환 [samples, time steps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_valid = x_valid.reshape((x_valid.shape[0], 1, x_valid.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

# GRU 모델 생성
model = Sequential()
model.add(GRU(50, return_sequences=True, input_shape=(1, x_train.shape[2])))
model.add(GRU(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 모델 훈련
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=100, batch_size=1, verbose=1)

# 모델 예측
train_predict = model.predict(x_train)
valid_predict = model.predict(x_valid)
test_predict = model.predict(x_test)



Epoch 1/100


348/348 [==============================] - 12s 11ms/step - loss: 4.9664 - mae: 1.2598 - val_loss: 1.9353 - val_mae: 0.7677
Epoch 2/100
348/348 [==============================] - 2s 5ms/step - loss: 2.5442 - mae: 0.9066 - val_loss: 1.8322 - val_mae: 0.7605
Epoch 3/100
348/348 [==============================] - 2s 6ms/step - loss: 2.0530 - mae: 0.8229 - val_loss: 1.9128 - val_mae: 0.7889
Epoch 4/100
348/348 [==============================] - 2s 5ms/step - loss: 1.9247 - mae: 0.8007 - val_loss: 1.7447 - val_mae: 0.8115
Epoch 5/100
348/348 [==============================] - 2s 6ms/step - loss: 1.9304 - mae: 0.7843 - val_loss: 1.7542 - val_mae: 0.7675
Epoch 6/100
348/348 [==============================] - 2s 5ms/step - loss: 1.9898 - mae: 0.7824 - val_loss: 1.7007 - val_mae: 0.7596
Epoch 7/100
348/348 [==============================] - 2s 6ms/step - loss: 1.7795 - mae: 0.7765 - val_loss: 1.7411 - val_mae: 0.7795
Epoch 8/100
348/348 [==============================] - 2s 5ms/s

In [1]:
# 모델 평가
mae = mean_absolute_error(y_test, test_predict)
r2 = r2_score(y_test, test_predict)

print(f'Test MAE: {mae:.3f}')
print(f'R2 Score: {r2:.3f}')

NameError: name 'mean_absolute_error' is not defined